## Introduction 

บทเรียนนี้จะครอบคลุม: 
- การเรียกใช้ฟังก์ชันคืออะไรและกรณีการใช้งานของมัน 
- วิธีสร้างการเรียกใช้ฟังก์ชันโดยใช้ OpenAI 
- วิธีการผสานการเรียกใช้ฟังก์ชันเข้ากับแอปพลิเคชัน 

## Learning Goals 

หลังจากทำบทเรียนนี้เสร็จสิ้น คุณจะรู้วิธีและเข้าใจ: 

- วัตถุประสงค์ของการใช้การเรียกใช้ฟังก์ชัน 
- การตั้งค่าการเรียกใช้ฟังก์ชันโดยใช้บริการ OpenAI 
- การออกแบบการเรียกใช้ฟังก์ชันที่มีประสิทธิภาพสำหรับกรณีการใช้งานของแอปพลิเคชันของคุณ


## การเข้าใจการเรียกใช้ฟังก์ชัน

สำหรับบทเรียนนี้ เราต้องการสร้างฟีเจอร์สำหรับสตาร์ทอัพด้านการศึกษาของเราที่อนุญาตให้ผู้ใช้ใช้แชทบอทเพื่อค้นหาหลักสูตรทางเทคนิค เราจะแนะนำหลักสูตรที่เหมาะสมกับระดับทักษะ บทบาทปัจจุบัน และเทคโนโลยีที่สนใจ

เพื่อให้เสร็จสมบูรณ์ เราจะใช้การผสมผสานของ:
 - `OpenAI` เพื่อสร้างประสบการณ์แชทสำหรับผู้ใช้
 - `Microsoft Learn Catalog API` เพื่อช่วยผู้ใช้ค้นหาหลักสูตรตามคำขอของผู้ใช้
 - `Function Calling` เพื่อรับคำถามของผู้ใช้และส่งไปยังฟังก์ชันเพื่อทำคำขอ API

เพื่อเริ่มต้น มาดูว่าทำไมเราถึงต้องการใช้การเรียกใช้ฟังก์ชันตั้งแต่แรก:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # รับคำตอบใหม่จาก GPT ที่สามารถเห็นการตอบกลับของฟังก์ชัน

print(second_response.choices[0].message)


### ทำไมต้องเรียกใช้ฟังก์ชัน

ถ้าคุณได้เรียนบทเรียนอื่น ๆ ในคอร์สนี้แล้ว คุณน่าจะเข้าใจพลังของการใช้โมเดลภาษาขนาดใหญ่ (LLMs) แล้ว หวังว่าคุณจะเห็นข้อจำกัดบางอย่างของพวกมันด้วยเช่นกัน

การเรียกใช้ฟังก์ชันเป็นคุณสมบัติของบริการ OpenAI ที่ออกแบบมาเพื่อแก้ไขปัญหาดังต่อไปนี้:

รูปแบบการตอบกลับที่ไม่สม่ำเสมอ:
- ก่อนการเรียกใช้ฟังก์ชัน การตอบกลับจากโมเดลภาษาขนาดใหญ่จะไม่มีโครงสร้างและไม่สม่ำเสมอ นักพัฒนาต้องเขียนโค้ดยืนยันความถูกต้องที่ซับซ้อนเพื่อจัดการกับความแตกต่างแต่ละแบบในผลลัพธ์

การรวมข้อมูลภายนอกที่จำกัด:
- ก่อนคุณสมบัตินี้ การนำข้อมูลจากส่วนอื่นของแอปพลิเคชันมาใช้ในบริบทของแชทเป็นเรื่องยาก

ด้วยการทำให้รูปแบบการตอบกลับเป็นมาตรฐานและเปิดใช้งานการรวมข้อมูลภายนอกได้อย่างราบรื่น การเรียกใช้ฟังก์ชันช่วยให้ง่ายต่อการพัฒนาและลดความจำเป็นในการเขียนตรรกะการยืนยันเพิ่มเติม

ผู้ใช้ไม่สามารถรับคำตอบเช่น "สภาพอากาศปัจจุบันในสตอกโฮล์มเป็นอย่างไร?" ได้ เนื่องจากโมเดลถูกจำกัดด้วยเวลาที่ข้อมูลถูกฝึกมา

มาดูตัวอย่างด้านล่างที่แสดงปัญหานี้:

สมมติว่าเราต้องการสร้างฐานข้อมูลข้อมูลนักเรียนเพื่อที่เราจะได้แนะนำหลักสูตรที่เหมาะสมให้กับพวกเขา ด้านล่างนี้เรามีคำอธิบายของนักเรียนสองคนที่มีข้อมูลคล้ายกันมาก


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

เราต้องการส่งสิ่งนี้ไปยัง LLM เพื่อวิเคราะห์ข้อมูล ซึ่งต่อไปสามารถนำไปใช้ในแอปพลิเคชันของเราเพื่อส่งไปยัง API หรือเก็บไว้ในฐานข้อมูล

มาสร้างพรอมต์สองชุดที่เหมือนกันซึ่งเราจะสั่งให้ LLM ทราบว่าข้อมูลใดที่เราสนใจ:


เราต้องการส่งสิ่งนี้ไปยัง LLM เพื่อวิเคราะห์ส่วนที่สำคัญต่อผลิตภัณฑ์ของเรา ดังนั้นเราสามารถสร้างพรอมต์สองชุดที่เหมือนกันเพื่อสั่งงาน LLM ได้:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


หลังจากสร้างพรอมต์ทั้งสองนี้แล้ว เราจะส่งพวกมันไปยัง LLM โดยใช้ `openai.ChatCompletion` เราจะเก็บพรอมต์ไว้ในตัวแปร `messages` และกำหนดบทบาทเป็น `user` ซึ่งเป็นการเลียนแบบข้อความจากผู้ใช้ที่เขียนถึงแชทบอท


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

ตอนนี้เราสามารถส่งคำขอทั้งสองไปยัง LLM และตรวจสอบการตอบกลับที่เราได้รับได้แล้ว


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

แม้ว่าคำสั่งจะเหมือนกันและคำอธิบายจะคล้ายกัน แต่เราสามารถได้รูปแบบที่แตกต่างกันของคุณสมบัติ `Grades`

หากคุณรันเซลล์ข้างต้นหลายครั้ง รูปแบบอาจเป็น `3.7` หรือ `3.7 GPA`

นี่เป็นเพราะ LLM รับข้อมูลที่ไม่มีโครงสร้างในรูปแบบของคำสั่งที่เขียนและส่งคืนข้อมูลที่ไม่มีโครงสร้างเช่นกัน เราจำเป็นต้องมีรูปแบบที่มีโครงสร้างเพื่อที่เราจะได้รู้ว่าจะคาดหวังอะไรเมื่อจัดเก็บหรือใช้ข้อมูลนี้

โดยการใช้การเรียกฟังก์ชัน เราสามารถมั่นใจได้ว่าเราจะได้รับข้อมูลที่มีโครงสร้างกลับมา เมื่อใช้การเรียกฟังก์ชัน LLM จะไม่เรียกหรือรันฟังก์ชันใด ๆ จริง ๆ แต่เราจะสร้างโครงสร้างให้ LLM ปฏิบัติตามสำหรับการตอบกลับ จากนั้นเราจะใช้การตอบกลับที่มีโครงสร้างเหล่านั้นเพื่อรู้ว่าจะรันฟังก์ชันใดในแอปพลิเคชันของเรา


![แผนภาพการไหลของการเรียกฟังก์ชัน](../../../../translated_images/Function-Flow.083875364af4f4bb.th.png)


เราสามารถนำสิ่งที่ฟังก์ชันส่งกลับมาแล้วส่งกลับไปยัง LLM ได้ จากนั้น LLM จะตอบกลับโดยใช้ภาษาธรรมชาติเพื่อตอบคำถามของผู้ใช้


### กรณีการใช้งานสำหรับการเรียกฟังก์ชัน

**การเรียกใช้เครื่องมือภายนอก**  
แชทบอทมีความสามารถยอดเยี่ยมในการให้คำตอบต่อคำถามจากผู้ใช้ โดยการใช้การเรียกฟังก์ชัน แชทบอทสามารถใช้ข้อความจากผู้ใช้เพื่อทำงานบางอย่างให้เสร็จสมบูรณ์ ตัวอย่างเช่น นักเรียนสามารถขอให้แชทบอท "ส่งอีเมลถึงอาจารย์ของฉันบอกว่าฉันต้องการความช่วยเหลือเพิ่มเติมในวิชานี้" ซึ่งสามารถทำการเรียกฟังก์ชัน `send_email(to: string, body: string)`

**สร้างคำสั่ง API หรือฐานข้อมูล**  
ผู้ใช้สามารถค้นหาข้อมูลโดยใช้ภาษาธรรมชาติที่ถูกแปลงเป็นคำสั่งค้นหาหรือคำขอ API ที่มีรูปแบบ ตัวอย่างเช่น ครูที่ขอว่า "ใครคือผู้ที่ทำการบ้านชิ้นสุดท้ายเสร็จแล้ว" ซึ่งอาจเรียกฟังก์ชันชื่อ `get_completed(student_name: string, assignment: int, current_status: string)`

**การสร้างข้อมูลที่มีโครงสร้าง**  
ผู้ใช้สามารถนำข้อความหรือไฟล์ CSV มาใช้ LLM เพื่อสกัดข้อมูลสำคัญออกมา ตัวอย่างเช่น นักเรียนสามารถแปลงบทความวิกิพีเดียเกี่ยวกับข้อตกลงสันติภาพเพื่อสร้างบัตรแฟลช AI ได้ ซึ่งสามารถทำได้โดยใช้ฟังก์ชันที่ชื่อ `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. การสร้างการเรียกฟังก์ชันแรกของคุณ

กระบวนการสร้างการเรียกฟังก์ชันประกอบด้วย 3 ขั้นตอนหลัก:
1. เรียกใช้ Chat Completions API พร้อมกับรายการฟังก์ชันของคุณและข้อความจากผู้ใช้
2. อ่านการตอบกลับของโมเดลเพื่อดำเนินการ เช่น การเรียกใช้ฟังก์ชันหรือ API
3. ทำการเรียกอีกครั้งไปยัง Chat Completions API พร้อมกับผลลัพธ์จากฟังก์ชันของคุณเพื่อใช้ข้อมูลนั้นในการสร้างการตอบกลับให้กับผู้ใช้


![การไหลของการเรียกฟังก์ชัน](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.th.png)


### องค์ประกอบของการเรียกฟังก์ชัน

#### การป้อนข้อมูลของผู้ใช้

ขั้นตอนแรกคือการสร้างข้อความของผู้ใช้ ซึ่งสามารถกำหนดค่าได้แบบไดนามิกโดยการรับค่าจากการป้อนข้อความ หรือคุณสามารถกำหนดค่าที่นี่ได้ หากนี่เป็นครั้งแรกที่คุณทำงานกับ Chat Completions API เราจำเป็นต้องกำหนด `role` และ `content` ของข้อความ

`role` สามารถเป็นได้ทั้ง `system` (การสร้างกฎ), `assistant` (โมเดล) หรือ `user` (ผู้ใช้ปลายทาง) สำหรับการเรียกฟังก์ชัน เราจะกำหนดเป็น `user` และตัวอย่างคำถามหนึ่งข้อ


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### การสร้างฟังก์ชัน

ต่อไปเราจะกำหนดฟังก์ชันและพารามิเตอร์ของฟังก์ชันนั้น เราจะใช้เพียงฟังก์ชันเดียวที่ชื่อว่า `search_courses` แต่คุณสามารถสร้างฟังก์ชันหลายฟังก์ชันได้

**สำคัญ** : ฟังก์ชันจะถูกรวมอยู่ในข้อความระบบสำหรับ LLM และจะถูกนับรวมในจำนวนโทเค็นที่คุณมีอยู่ด้วย


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**คำนิยาม**

โครงสร้างการกำหนดฟังก์ชันมีหลายระดับ โดยแต่ละระดับมีคุณสมบัติของตัวเอง นี่คือการแยกโครงสร้างแบบซ้อน:

**คุณสมบัติฟังก์ชันระดับบนสุด:**

`name` - ชื่อของฟังก์ชันที่เราต้องการให้เรียกใช้

`description` - คำอธิบายเกี่ยวกับวิธีการทำงานของฟังก์ชัน ที่นี่สำคัญที่จะต้องชัดเจนและเฉพาะเจาะจง

`parameters` - รายการของค่าต่าง ๆ และรูปแบบที่คุณต้องการให้โมเดลสร้างขึ้นในคำตอบ

**คุณสมบัติวัตถุพารามิเตอร์:**

`type` - ประเภทข้อมูลของวัตถุพารามิเตอร์ (โดยปกติจะเป็น "object")

`properties` - รายการของค่าที่เฉพาะเจาะจงที่โมเดลจะใช้สำหรับคำตอบ

**คุณสมบัติพารามิเตอร์แต่ละตัว:**

`name` - กำหนดโดยคีย์ของคุณสมบัติอย่างชัดเจน (เช่น "role", "product", "level")

`type` - ประเภทข้อมูลของพารามิเตอร์เฉพาะนี้ (เช่น "string", "number", "boolean")

`description` - คำอธิบายของพารามิเตอร์เฉพาะนี้

**คุณสมบัติเพิ่มเติม (ไม่บังคับ):**

`required` - อาร์เรย์ที่ระบุว่าพารามิเตอร์ใดบ้างที่จำเป็นสำหรับการเรียกใช้ฟังก์ชันให้เสร็จสมบูรณ์


### การเรียกใช้ฟังก์ชัน  
หลังจากกำหนดฟังก์ชันแล้ว ตอนนี้เราจำเป็นต้องรวมฟังก์ชันนั้นในการเรียกใช้ Chat Completion API เราทำได้โดยการเพิ่ม `functions` ลงในคำขอ ในกรณีนี้คือ `functions=functions`  

นอกจากนี้ยังมีตัวเลือกในการตั้งค่า `function_call` เป็น `auto` ซึ่งหมายความว่าเราจะปล่อยให้ LLM ตัดสินใจว่าควรเรียกใช้ฟังก์ชันใดตามข้อความของผู้ใช้ แทนที่จะกำหนดเอง


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

ตอนนี้เรามาดูการตอบกลับและดูว่ามันถูกจัดรูปแบบอย่างไร:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

คุณจะเห็นว่าชื่อของฟังก์ชันถูกเรียกใช้และจากข้อความของผู้ใช้ LLM สามารถค้นหาข้อมูลให้ตรงกับอาร์กิวเมนต์ของฟังก์ชันได้


## 3.การรวมการเรียกฟังก์ชันเข้ากับแอปพลิเคชัน

หลังจากที่เราได้ทดสอบการตอบกลับที่จัดรูปแบบจาก LLM แล้ว ตอนนี้เราสามารถรวมสิ่งนี้เข้ากับแอปพลิเคชันได้

### การจัดการลำดับการทำงาน

เพื่อรวมสิ่งนี้เข้ากับแอปพลิเคชันของเรา ให้ทำตามขั้นตอนดังต่อไปนี้:

อันดับแรก ให้ทำการเรียกใช้บริการ OpenAI และเก็บข้อความไว้ในตัวแปรที่ชื่อว่า `response_message`


In [ ]:
response_message = response.choices[0].message

ตอนนี้เราจะกำหนดฟังก์ชันที่จะเรียกใช้ Microsoft Learn API เพื่อรับรายการหลักสูตร:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



ในฐานะแนวทางปฏิบัติที่ดีที่สุด เราจะตรวจสอบว่ารุ่นต้องการเรียกใช้ฟังก์ชันหรือไม่ หลังจากนั้น เราจะสร้างหนึ่งในฟังก์ชันที่มีอยู่และจับคู่กับฟังก์ชันที่ถูกเรียกใช้  
จากนั้นเราจะนำอาร์กิวเมนต์ของฟังก์ชันมาแมปกับอาร์กิวเมนต์จาก LLM  

สุดท้าย เราจะต่อข้อความการเรียกฟังก์ชันและค่าที่ถูกส่งกลับโดยข้อความ `search_courses` ซึ่งจะให้ข้อมูลทั้งหมดที่ LLM ต้องการ  
เพื่อตอบกลับผู้ใช้โดยใช้ภาษาธรรมชาติ


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



ตอนนี้เราจะส่งข้อความที่อัปเดตไปยัง LLM เพื่อที่เราจะได้รับการตอบกลับเป็นภาษาธรรมชาติแทนการตอบกลับในรูปแบบ JSON ของ API


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

เยี่ยมมาก! เพื่อสานต่อการเรียนรู้เกี่ยวกับ OpenAI Function Calling คุณสามารถสร้างได้ที่: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - พารามิเตอร์เพิ่มเติมของฟังก์ชันที่อาจช่วยให้ผู้เรียนค้นหาคอร์สเพิ่มเติมได้ คุณสามารถดูพารามิเตอร์ API ที่มีได้ที่นี่: 
 - สร้างการเรียกฟังก์ชันอีกตัวที่รับข้อมูลเพิ่มเติมจากผู้เรียน เช่น ภาษาพื้นเมืองของพวกเขา 
 - สร้างการจัดการข้อผิดพลาดเมื่อการเรียกฟังก์ชันและ/หรือการเรียก API ไม่ส่งคืนคอร์สที่เหมาะสมใดๆ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษาอัตโนมัติ [Co-op Translator](https://github.com/Azure/co-op-translator) แม้เราจะพยายามให้ความถูกต้องสูงสุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลโดยผู้เชี่ยวชาญมนุษย์ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดใด ๆ ที่เกิดจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
